# TL06: Modelos de Lenguaje (LLMs)

Este notebook introduce conceptos fundamentales sobre:
- **Generación de texto** con modelos de lenguaje
- **Estrategias de generación** (greedy, sampling, beam search)
- **Ingeniería de prompts** (zero-shot, few-shot, chain-of-thought)
- **Benchmarking** de LLMs

## Documentación de referencia:
- Transformers: TL02 (Get started), TL03 (Clases base e inferencia)
- PyTorch: TL01
- Datasets y Evaluate: TL04


## 2. Generación de texto

En esta sección veremos cómo generar texto usando el método `generate()` de transformers.

Cargamos el modelo **Qwen/Qwen3-0.6B** como ejemplo. Este es un modelo causal de lenguaje (CausalLM) que puede generar texto de forma autónoma.


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name) # padding_side="left"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Tokenizamos el texto de entrada. El tokenizer convierte el texto en tokens numéricos (`input_ids`) que el modelo puede procesar. También genera una `attention_mask` que indica qué tokens son reales (1) y cuáles son padding (0).


In [2]:
model_inputs = tokenizer(["Lista de colores: rojo, azul"], return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[43617,   409,  1375,  4589,    25,   926,  7305,    11, 12376,   360]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Generamos texto usando `model.generate()`. El modelo predice los siguientes tokens basándose en la entrada proporcionada.

**Nota importante**: Observa que cada palabra puede corresponder a uno o más tokens. Por ejemplo, "azul" se tokeniza como los tokens 12376 y 360, y la coma produce el token 11.


In [3]:
generated_ids = model.generate(**model_inputs)
generated_ids

tensor([[43617,   409,  1375,  4589,    25,   926,  7305,    11, 12376,   360,
            11, 73161,    11, 89547,    11,  4184,   299,    11,  1079,   277,
         21782,    11,   926,  7305,    11, 73161,    11, 12376,   360,    11]],
       device='cuda:0')

Decodificamos los IDs generados de vuelta a texto legible usando `batch_decode()`. El parámetro `skip_special_tokens=True` elimina tokens especiales como `<pad>`, `<eos>`, etc.


In [4]:
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'Lista de colores: rojo, azul, verde, blanco, negro, amarillo, rojo, verde, azul,'

### Configuración de generación

Cada modelo tiene una configuración de generación por defecto (`generation_config`) que incluye parámetros como:
- `temperature`: Controla la aleatoriedad (0.6 por defecto)
- `top_k`: Limita la selección a los k tokens más probables (20 por defecto)
- `top_p`: Muestreo de núcleo (nucleus sampling) con probabilidad acumulada (0.95 por defecto)
- `do_sample`: Si es `True`, usa muestreo; si es `False`, usa decodificación greedy


In [5]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

### Parámetros de generación

Podemos modificar los parámetros de generación de dos formas:
1. **Parámetros de `generation_config`**: `max_length`, `max_new_tokens`, `temperature`, etc.
2. **kwargs de `generate()`**: Pueden incluir parámetros ad hoc de `generation_config` y parámetros específicos del modelo

**Ejemplo 1**: Limitamos la longitud máxima de la secuencia generada con `max_length=36`.


In [6]:
generated_ids = model.generate(**model_inputs, max_length=36)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'Lista de colores: rojo, azul, verde, blanco, naranja, marrón, rosa, amarillo, negro, cian, verde'

**Ejemplo 2**: Aumentamos la temperatura a 1.0 para generar texto más diverso y creativo. Valores más altos de temperatura aumentan la aleatoriedad en la generación.


In [7]:
generated_ids = model.generate(**model_inputs, temperature=1.0)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'Lista de colores: rojo, azul, verde, blanco, naranja, violeta, gris, amarillo.\n\nEscri'

## 3. Estrategias de generación

Existen diferentes métodos de decodificación para generar texto:

1. **Greedy search**: Selecciona siempre el token más probable en cada paso. Es determinista pero puede producir repeticiones.
2. **Sampling**: Muestrea el siguiente token de la distribución de probabilidades. Produce texto más diverso.
3. **Beam search**: Explora múltiples secuencias simultáneamente (haz de tamaño `num_beams`) y selecciona la secuencia más probable. Equilibra entre calidad y diversidad.

A continuación comparamos los tres métodos con el mismo prompt.


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer; model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name) # padding_side="left"
inputs = tokenizer(["Lista de colores: rojo, azul"], return_tensors="pt").to(model.device)

## 4. Ingeniería de prompts

La ingeniería de prompts consiste en diseñar prompts que ayuden al modelo a producir los resultados deseados. Veremos tres técnicas principales:

### 4.1. Ejemplo básico de prompting

En este ejemplo, usamos un prompt estructurado para realizar una tarea de clasificación de sentimiento.


In [9]:
outputs = model.generate(**inputs, max_new_tokens=50)
print("\nGreedy:\n", tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, num_beams=1)
print("\nSampling:\n",tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
outputs = model.generate(**inputs, max_new_tokens=50, num_beams=2)
print("\nBeam search:\n", tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


Greedy:
 Lista de colores: rojo, azul, verde, naranja, gris, blanco, rojo, azul, verde, gris, blanco, rojo, azul, verde, gris, blanco, rojo, azul, verde, gris, blanco, rojo,

Sampling:
 Lista de colores: rojo, azul, verde, naranja, marrón, amarillo, blanco, negro, gris, lila, violeta, gris, azul, marrón, amarillo, blanco, negro, gris, lila, violet

Beam search:
 Lista de colores: rojo, azul, verde, blanco, gris, naranja, amarillo, marrón, rojo, naranja, amarillo, marrón, verde, azul, blanco, gris, naranja, amarillo


### 4.2. Zero-shot prompting

El **zero-shot prompting** consiste en usar un buen prompt sin proporcionar ejemplos previos. El modelo debe inferir la tarea únicamente a partir de las instrucciones del prompt.

En este ejemplo, extraemos una fecha de un texto sin mostrar ejemplos previos.


In [10]:
from transformers import pipeline
pipeline = pipeline(model="Qwen/Qwen3-4B", device_map="auto")
prompt = """
Tarea: Clasifica el texto en positivo o negativo.
Texto: Es mi película favorita.
Resultado:"""
print(pipeline(prompt, max_new_tokens=3, do_sample=True, top_k=10)[0]['generated_text'])

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cuda:0



Tarea: Clasifica el texto en positivo o negativo.
Texto: Es mi película favorita.
Resultado: Positivo


### 4.3. Few-shot prompting

El **few-shot prompting** consiste en proporcionar ejemplos específicos de la tarea antes de la entrada real. Esto ayuda al modelo a entender mejor el formato y el patrón esperado.

En este ejemplo, mostramos un ejemplo de extracción de fecha con formato específico (DD-MM-YYYY) antes de pedirle al modelo que extraiga otra fecha con el mismo formato.


In [11]:
prompt = """
Texto: El primer debate presidencial televisado en Estados Unidos tuvo lugar el 28 de septiembre de 1960.
Fecha:"""
print(pipeline(prompt, max_new_tokens=12, do_sample=True, top_k=10)[0]['generated_text'])


Texto: El primer debate presidencial televisado en Estados Unidos tuvo lugar el 28 de septiembre de 1960.
Fecha: 28 de septiembre de 1960


### 4.4. Chain-of-Thought (CoT) prompting

El **Chain-of-Thought (CoT)** consiste en emplear una serie de pasos de razonamiento que ayudan al modelo a "pensar" paso a paso para hallar la respuesta deseada. Esto es especialmente útil para problemas que requieren razonamiento matemático o lógico.

En este ejemplo, el modelo resuelve un problema de matemáticas siguiendo los pasos de razonamiento proporcionados en el prompt.


In [12]:
prompt = """
Texto: El primer humano que salió al espacio y orbitó la Tierra lo hizo el 12 de abril de 1961.
Fecha: 12-04-1961
Texto: El primer debate presidencial televisado en Estados Unidos tuvo lugar el 28 de septiembre de 1960.
Fecha:"""
print(pipeline(prompt, max_new_tokens=12, do_sample=True, top_k=10)[0]['generated_text'])


Texto: El primer humano que salió al espacio y orbitó la Tierra lo hizo el 12 de abril de 1961.
Fecha: 12-04-1961
Texto: El primer debate presidencial televisado en Estados Unidos tuvo lugar el 28 de septiembre de 1960.
Fecha: 28-09-1960



## 5. Benchmarking

El benchmarking de LLMs consiste en comparar el rendimiento de diferentes modelos de lenguaje usando colecciones estandarizadas de tareas.

### Recursos de benchmarking:
- **HF Leaderboards**: Clasificaciones y evaluaciones de Hugging Face
- **Artificial Analysis**: Análisis del estado actual de la IA y LLMs
- **LiveBench**: Leaderboard de LLMs sin "contaminación" del test set

### MMLU-ProX

**MMLU-ProX** es una versión multilíngüe y extendida de MMLU (Measuring Massive Multitask Language Understanding), un benchmark que evalúa la comprensión de lenguaje en múltiples dominios mediante preguntas de opción múltiple.

En este ejemplo, cargamos el dataset MMLU-ProX en español y probamos el modelo con una pregunta del test.


In [13]:
prompt = """Let's go through this step-by-step:
1. You start with 15 muffins.
2. You eat 2 muffins, leaving you with 13 muffins.
3. You give 5 muffins to your neighbor, leaving you with 8 muffins.
4. Your partner buys 6 more muffins, bringing the total number of muffins to 14.
5. Your partner eats 2 muffins, leaving you with 12 muffins.
If you eat 6 muffins, how many are left?"""
print(pipeline(prompt, max_new_tokens=12, num_beams=10)[0]['generated_text'])

Let's go through this step-by-step:
1. You start with 15 muffins.
2. You eat 2 muffins, leaving you with 13 muffins.
3. You give 5 muffins to your neighbor, leaving you with 8 muffins.
4. Your partner buys 6 more muffins, bringing the total number of muffins to 14.
5. Your partner eats 2 muffins, leaving you with 12 muffins.
If you eat 6 muffins, how many are left? 12 - 6 = 6 muffins left


### Ejemplo de evaluación con MMLU-ProX

Probamos el modelo con una pregunta de opción múltiple del dataset MMLU-ProX. Usamos un prompt estructurado que incluye todas las opciones y le pedimos al modelo que responda con una sola letra.


In [16]:
from datasets import load_dataset; from transformers import pipeline
pipe = pipeline(model="Qwen/Qwen3-4B", device_map="auto")
ds = load_dataset("li-lab/MMLU-ProX", "es", split="test")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


README.md: 0.00B [00:00, ?B/s]

es/validation-00000-of-00001.parquet:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

es/test-00000-of-00001.parquet:   0%|          | 0.00/5.47M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/70 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11759 [00:00<?, ? examples/s]

In [17]:
prompt = """
Texto: Los organismos reguladores de publicidad típicos sugieren, por ejemplo, que los anuncios no deben: fomentar __
A: Prácticas seguras, Miedo, Celos, Trivial
B: Prácticas inseguras, Angustia, Alegría, Trivial
C: Prácticas seguras, Deseos, Celos, Trivial
D: Prácticas seguras, Angustia, Miedo, Trivial
E: Prácticas inseguras, Deseos, Celos, Trivial
F: Prácticas seguras, Angustia, Celos, Trivial
G: Prácticas seguras, Deseos, Miedo, Trivial
H: Prácticas inseguras, Deseos, Miedo, Trivial
I: Prácticas inseguras, Angustia, Miedo, Trivial
Tarea: indica con una sola letra la opción correcta, sin explicación
Respuesta:
"""
print(pipe(prompt, max_new_tokens=2, num_beams=2)[0]['generated_text'])


Texto: Los organismos reguladores de publicidad típicos sugieren, por ejemplo, que los anuncios no deben: fomentar __
A: Prácticas seguras, Miedo, Celos, Trivial
B: Prácticas inseguras, Angustia, Alegría, Trivial
C: Prácticas seguras, Deseos, Celos, Trivial
D: Prácticas seguras, Angustia, Miedo, Trivial
E: Prácticas inseguras, Deseos, Celos, Trivial
F: Prácticas seguras, Angustia, Celos, Trivial
G: Prácticas seguras, Deseos, Miedo, Trivial
H: Prácticas inseguras, Deseos, Miedo, Trivial
I: Prácticas inseguras, Angustia, Miedo, Trivial
Tarea: indica con una sola letra la opción correcta, sin explicación
Respuesta:
F

